In [ ]:
import pandas as pd
import numpy as np
%config Completer.use_jedi = False
from PIL import Image
import random
from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw
import io

In [ ]:
kills = pd.read_csv("../input/pubg-match-deaths/deaths/kill_match_stats_final_0.csv")
matches = pd.read_csv("../input/pubg-match-deaths/aggregate/agg_match_stats_0.csv")

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(15,15)})
ax=sns.countplot(y="killed_by",data=kills)

In [ ]:
zero_kill_deaths=matches[matches["player_kills"]==0]
zero_kill_assist_deaths=zero_kill_deaths[zero_kill_deaths["player_assists"]==0]
zero_kill_assist_knock_deaths=zero_kill_assist_deaths[zero_kill_assist_deaths["player_dbno"]==0]
zero_kill_assist_knock_deaths.shape
bots_death_stat=zero_kill_assist_knock_deaths[zero_kill_assist_knock_deaths["player_dist_ride"]<50]
bots_death_stat.shape

In [ ]:
erangel_img=Image.open("../input/pubg-match-deaths/erangel.jpg")
erangel_img

In [ ]:
bots_position=pd.DataFrame(columns=["position_x","position_y"])
match_list=random.sample(set(bots_death_stat["match_id"]),100)
for x in range(100):
    selected_match_stat=bots_death_stat[bots_death_stat["match_id"]==match_list[x]]
    for idx,match in selected_match_stat.iterrows():
        for ldx,kill in kills.iterrows():
            if match["match_id"]==kill["match_id"] & match["player_name"]==kill["victim_name"]:
                bots_position.append({"position_x":kill["victim_position_x"],"position_y":kill["victim_position_y"]},ignore_index=True)


In [ ]:
def adjust_position(df,m="e"):
    scaler=4096/800000
    if m!="e":
        scaler/=1.25
    columns=["position_x","position_y"]
    for col in columns:
        df[col]=df[col]*scaler
    return df

In [ ]:
scaled_position_bot=adjust_position(bots_position)

In [ ]:
reds_color = np.zeros((100, 4))
reds_color[:, 0] = np.linspace(0, 1, 100)
reds_color[:, 3] = np.linspace(0.4, 1, 100)
reds_color[:10] = 0
alpha_reds_cmap = mcolors.LinearSegmentedColormap.from_list('alpha_reds', colors=reds_color, N=1024, gamma=1)

greens_color = np.zeros((100, 4))
greens_color[:, 1] = np.linspace(0, 1, 100)
greens_color[:, 3] = np.linspace(0.4, 1, 100)
greens_color[:10] = 0
alpha_greens_cmap = mcolors.LinearSegmentedColormap.from_list('alpha_greens', colors=greens_color, N=1024, gamma=1)
def generate_heatmap(df,
                     img,
                     n_grid=100,
                     bw=200,
                     kernel='epanechnikov',
                     cmap=plt.cm.Reds,
                     alpha=None,
                     ax=None,
                    ):
    x_mesh = np.linspace(0, 4096, n_grid)
    y_mesh = np.linspace(0, 4096, n_grid)
    X, Y = np.meshgrid(x_mesh, y_mesh)
    xy = np.vstack([X.ravel(), Y.ravel()]).T

    kde = KernelDensity(kernel=kernel, bandwidth=bw).fit(df.to_numpy())
    log_density = kde.score_samples(xy)
    log_density = log_density.reshape((n_grid, n_grid))
    density = np.exp(log_density)

    if ax is None:
        fig, ax = plt.subplots(figsize=(16, 16))

    plt.gca().invert_yaxis()
    ax.contourf(X, Y, density, alpha=alpha, cmap=cmap)
    ax.imshow(img)
    return fig, ax

In [ ]:
fig, ax = generate_heatmap(scaled_position_bot[["position_x","position_y"]], erangel_img, bw=200, cmap=alpha_reds_cmap)